In [ ]:
import numpy as np

import GPyOpt
import GPy

import sys
sys.path.append("..")
sys.path.append("csaf/core")
sys.path.append("../../examples/f16/")
sys.path.append("../../examples/f16/components")

from f16lib.acas import AcasScenario, AcasShieldScenario, AcasScenarioViewer

from components import fgnetfdm, fgintruder

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'


## Construct the F16 Scenario

In [ ]:
def intruder_airspeed(t):
    #return min(550.0 + t * 10.0, 800.0)
    if t <= 7.0:
        return 800.0
    else: 
        return 1000.0


def air_collision_condition(ctraces):
        """ground collision premature termination condition
        """
        # get the aircraft states
        sa, sb, sc = ctraces['plant']['states'], ctraces['intruder_plant']['states'], ctraces['balloon']['states']
        if sa and sb and sc:
            # look at distance between last state
            dab =  (np.linalg.norm(np.array(sa[-1][9:11]) - np.array(sb[-1][9:11]))) 
            dac = (np.linalg.norm(np.array(sa[-1][9:11]) - np.array(sc[-1][9:11]))) 
            return dab < 250.0 or dac < 250.0
        


def get_system(states):
    """construct a scenario and system given the configuration coordinates (states)"""
    # scenario
    scen = AcasScenario(
        [-3000.0, 12000], # balloon position
        #[12000.0, -12000],
        800.0, # ownship airspeed
        ((0.0, 21000.0, 1000.0),), # own waypoints
        ((*states[:2], 1000.0),), # intruder waypoints -- none for now
        intruder_velocity = intruder_airspeed
    )
    # scenario, system
    return scen, scen.create_system(
        [*states[:2], # relative position
         states[2], # relative heading
         states[3]]) # relative airspeed


def objective_function(states) -> float:
    """obj: configuration space -> real number"""
    # run simulation
    _, sys = get_system(states)
    trajs, p = sys.simulate_tspan((0.0, 30.0), return_passed=True)
    
    # get distances between ownship and intruder
    intruder_pos = np.array(trajs['intruder_plant'].states)[:, 9:11]
    ownship_pos = np.array(trajs['plant'].states)[:, 9:11]
    rel_pos = intruder_pos - ownship_pos
    
    # get distances between ownship and balloon
    dists = np.linalg.norm(rel_pos, axis=1)
    ballon_dists = ownship_pos - np.tile(np.array(trajs['balloon'].states)[-1, 9:11][:], (len(ownship_pos), 1))
    bdists = np.linalg.norm(ballon_dists, axis=1)
    
    # get objective (min distance to obstacles)
    print("OBJECTIVE (min distance): ", min(np.hstack((dists, bdists))), ",", np.sqrt(min(dists) * min(bdists)))
    # geometric mean of min dists
    return np.sqrt(min(dists) * min(bdists))
    # minimum distance
    #return min(np.hstack((dists, bdists)))


def objective_function_opt(x):
    """GPyOpt Objective"""
    return np.array([objective_function(xi) for xi in x])

## Formulate the Optimization Problem

In [ ]:
# describe initial states (via bounds)
bounds = [{'name': 'x_pos', 'type': 'continuous', 'domain': (-10000,10000)},
          {'name': 'y_pos', 'type': 'continuous', 'domain': (-10000,10000)},
          {'name': 'theta', 'type': 'continuous', 'domain': (-np.pi,np.pi)},
          {'name': 'speed', 'type': 'continuous', 'domain': (0,0)}]

# additional constraints to reduce the initial states
constraints = [
    # keep intruder initial position at least 7000 ft away
    {'name': 'constr_1', 'constraint': '-((x[:, 1]**2 + x[:, 0]**2) - 7000**2)'},
    # keep intruder "pointed at" ownship, plus/minus 90 degrees
    # TODO: this doesn't look at all quadrants - debug?
    #{'name': 'constr_2', 'constraint': 'np.abs((np.pi + x[:, 2]) - np.arctan2(x[:, 1], x[:, 0])) - np.pi/2'}
]

In [ ]:
# get feasible region from initial states and constraints
feasible_region = GPyOpt.Design_space(space = bounds, constraints = constraints)

# generate initial designs
initial_design = GPyOpt.experiment_design.initial_design('random', feasible_region, 10)
# incorporate "good guesses"
initial_design = np.vstack((
    # "good guess" 1
    #[0.0, 8000, np.pi, 0.0], 
    # "good guess" 2
    [7000, 1000.0, -np.pi/2, -100.0], 
    initial_design))

In [ ]:
# get GPyOpt objective from function
objective = GPyOpt.core.task.SingleObjective(objective_function_opt)

In [ ]:
# custom kernel!
# we know that the relative heading angle is periodic, so we can set and fix it
# we know that the lengthscales and periods will be different along each natural axis, so we turn on ARD
# we know prior variance is low, so we can set low as well
k = GPy.kern.StdPeriodic(4,  # dimension
                         ARD1=True, ARD2=True, 
                         variance=1E-2,
                         period=[1E10, 1E10, 2*np.pi, 1E8], 
                         lengthscale=[200.0/3, 200.0/3, 0.06/3, 4.0/3])
k.period.fix()
k.lengthscale.fix()
#k.variance.fix()


In [ ]:
def cost(x):
    """weakly penalize searching too far away (this is optional)"""
    cost_f  = -np.atleast_2d(.001*x[:,0]**2 +.001*x[:,1]**2).T
    cost_df = -np.array([0.002*x[:,0],0.002*x[:,1]]).T
    return cost_f, cost_df

In [ ]:
# get GPModel from the covariance (kernel)
model = GPyOpt.models.GPModel(exact_feval=True,optimize_restarts=0,verbose=False,kernel=k)

# get the GPOpt acquisition optimizer
aquisition_optimizer = GPyOpt.optimization.AcquisitionOptimizer(feasible_region)

# get the type of acquisition
acquisition = GPyOpt.acquisitions.AcquisitionEI(model, feasible_region, optimizer=aquisition_optimizer)

# get the collection method
evaluator = GPyOpt.core.evaluators.Sequential(acquisition)

# get a cost model from the cost func
cost = GPyOpt.core.task.cost.CostModel(cost)

## Solve the Optimization Problem

In [ ]:
# build the modular Bayesian optimization model
bo = GPyOpt.methods.ModularBayesianOptimization(model, 
                                                feasible_region, 
                                                objective, 
                                                acquisition, 
                                                evaluator, 
                                                initial_design, 
                                                cost=None)

## Visualize the Optimal Configuration

In [ ]:
#idx = idxs[8]
#print(bo.X[idx])
#x_pt = bo.X[idx]


# collision
#x_pt = np.array([-6.28136243e+03,
#                 4.82438421e+03,
#                 1.13059323e+00,
#                 0.00000000e+00])

# recovery collision
#x_pt = np.array([-2.59438967e+03,  8.45941195e+03,  2.48404095e+00,  0.00000000e+00])

# succession of both vehicles
#x_pt = np.array([ 8.11003833e+03,  6.35794347e+03, -6.21766940e-01,  0.00000000e+00])

# success
#x_pt = np.array([-8.17529224e+03,
#                 9.97838802e+03,
#                 1.80759653e+00,
#                 0.00000000e+00])

# near collision
#x_pt = np.array([-5.81146488e+03,
#                 5.77646592e+03,
#                 1.56159655e+00,
#                 0.00000000e+00])

# no collision
x_pt = np.array([ 7.48737042e+03,
                  7.22693875e+03, 
                  -1.27871855e+00,
                  0.00000000e+00])


scen, sys = get_system(x_pt) # relative airspeed
trajs, p = sys.simulate_tspan((0.0, 20.0), return_passed=True, terminating_conditions_all=air_collision_condition)
viewer = AcasScenarioViewer(trajs, scen)
fig, ax = viewer.summary_plot()
plt.show()

In [ ]:
# Find the point of closest contact
ownship = trajs['plant']['states']
intruder = trajs['intruder_plant']['states']

d_min = np.inf
d_idx = 0
for idx in range(0,len(ownship)):
    pn_o = ownship[idx][9]
    pe_o = ownship[idx][10]
    pn_i = intruder[idx][9]
    pe_i = intruder[idx][10]
    d = np.sqrt((pn_o -pn_i)**2 + (pe_o - pe_i)**2)
    if d < d_min:
        d_min = d
        d_idx = idx

pn_0 = ownship[d_idx][9]
pe_0 = ownship[d_idx][10]
print(f"D_min={d_min}, idx={d_idx}, pn_0 = {pn_0},pe_0={pe_0}")

In [ ]:
# Place camera
import copy
camera = copy.deepcopy(trajs['balloon'])
for state in camera.states:
    state[9] = pn_0
    state[10] = pe_0
trajs['camera'] = camera

In [ ]:
f16a = fgnetfdm.FGNetFDM(h0_m=1500)
f16b = fgintruder.FGIntruder(callsign='F-16B', model_path='Aircraft/f16/Models/F-16.xml', fallback_model_index=516)
balloon = fgintruder.FGIntruder(callsign='Balloon', model_path='Aircraft/Hot-Air-Balloon/Models/Hot-Air-Balloon.xml', fallback_model_index=702)
camera = fgintruder.FGIntruder(callsign='Camera', model_path='Aircraft/ufo/Models/ufo.xml', fallback_model_index=712)

In [ ]:
f16a.start()
f16b.start()
balloon.start()
camera.start()

In [ ]:
# Properly prepare the intruders
f16a.set_trajs(trajs['plant'],trajs['llc'])
f16b.set_trajs(trajs['intruder_plant'],trajs['intruder_llc'])
# LLC for balloon doesn't matter
balloon.set_trajs(trajs['balloon'],trajs['llc'])
camera.set_trajs(trajs['camera'],trajs['llc'])

In [ ]:
f16a.reset()
f16b.reset()
balloon.reset()
camera.reset()

In [ ]:
# Run the simulation
f16a.simulate(delta_t=0.01,speed=0.9)
f16b.simulate(delta_t=0.5,speed=0.9)
balloon.simulate(delta_t=0.5,speed=0.9)
camera.simulate(delta_t=0.5,speed=0.9)

In [ ]:
f16a.stop()
f16b.stop()
balloon.stop()
camera.stop()